Imports

In [1]:
import requests

from arcgis.gis import GIS
from arcgis.features import manage_data

import numpy as np
import pandas as pd

import warnings
from urllib3.exceptions import InsecureRequestWarning
warnings.simplefilter("ignore", InsecureRequestWarning)

Connect to ArcGIS

In [2]:
gis = GIS("home")

Get my geometries and test layer

In [3]:
dd_id = "d37c3c2a6f1c4586baad82828bfc3c59"
dd_item = gis.content.get(dd_id)
geometries_layer = dd_item.layers[1]

# Item ID 0 is the output layer displayed in the dashboard
# COMMENTING OUT WHILE TESTING
#dashboard_layer = dd_item.layers[0]

test_id = "edb716da51bc4f7882d13d425ad08fd2"
test_item = gis.content.get(test_id)
dashboard_layer = test_item.layers[0]

Create spatial dfs from dashboard layer and geometries layer

In [4]:
dashboard_sdf = dashboard_layer.query().sdf
dashboard_sdf.shape

geometries_sdf = geometries_layer.query().sdf
geometries_sdf.shape

(4000, 19)

Get the goods from openFEMA API

In [5]:
api_url = r"https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$filter=fyDeclared ge 2013&$orderby=declarationDate desc"

response = requests.get(api_url)
data = response.json()
summaries_df = pd.DataFrame(data["DisasterDeclarationsSummaries"])

In summaries df, convert pseudo-date string fields to date fields...

In [6]:
date_columns = ["declarationDate", "incidentBeginDate", "incidentEndDate", "disasterCloseoutDate", "lastIAFilingDate", "lastRefresh"]

for dc in date_columns:
    summaries_df[dc] = pd.to_datetime(summaries_df[dc], errors="coerce")

In summaries df, add & calc full fips & full tribal code fields; add & calc covid & entity fields
* No I can't move these steps below checking for adds because the check for adds needs e.i Entity field

In [7]:
# Add & calc FIPS & Tribal FIPS fields
summaries_df["fipsFullCode"] = summaries_df["fipsStateCode"] + summaries_df["fipsCountyCode"]
summaries_df["fipsTribalCode"] = summaries_df["fipsStateCode"] + summaries_df["placeCode"]

# Add & calc COVID-19 field
summaries_df["COVID19"] = np.where(summaries_df["declarationTitle"].str.contains("COVID-19"), "Show only COVID-19", "Show only non-COVID-19")

# Add & calc Entity field
entity_conditions = [
    summaries_df["designatedArea"] == "Statewide",
    (summaries_df["designatedArea"] != "Statewide") & (summaries_df["fipsCountyCode"] == "000"),
    (summaries_df["designatedArea"] != "Statewide") & (summaries_df["fipsCountyCode"] != "000")
]

entity_values = ["State or Equivalent", "Tribal Area or Equivalent", "County or Equivalent"]

summaries_df["Entity"] = np.select(entity_conditions, entity_values)

Check whether there are adds (rows in Summaries not already in Dashboard)

In [8]:
multi_key = ["femaDeclarationString", "Entity"]

pre_adds = summaries_df.merge(dashboard_sdf[multi_key], on=multi_key, how="left", indicator=True)
adds_df = pre_adds[pre_adds["_merge"] == "left_only"].drop(columns=["_merge"])

adds_df.shape

(0, 32)

Set up vars & field mapping for use in parsing out out multi-key adds

In [9]:
state = "State or Equivalent"
county = "County or Equivalent"
tribal = "Tribal Area or Equivalent" 

state_field = "State_FIPS"
county_field = "Full_FIPS"
tribal_field1 = "AIANNHFP1"
tribal_field2 = "AIANNHFP2"
tribal_field3 = "AIANNHFP3"

key_fields_dict = {
    "State_FIPS": "fipsStateCode",
    "Full_FIPS": "fipsFullCode",
    "AIANNHFP1": "fipsTribalCode",
    "AIANNHFP2": "fipsTribalCode", # No idea why there are three of these...or why the Summaries sometimes use 2 and 3...
    "AIANNHFP3": "fipsTribalCode" # What a pain in the @$$...
}

Get the geometries for all multi-key combos

In [10]:
def get_geometries(entity, field):

    # Checking Dec-string-per-entity
    entity_adds_df = adds_df[adds_df["Entity"] == entity]

    # NEED to specify an empty string for at least one of the suffixes here;
    # Otherwise I have no "Entity" field and my Dashboard layer and adds feature collection won't have matching fields
    merged = entity_adds_df.merge(geometries_sdf, left_on=[key_fields_dict[field]], right_on=field, suffixes=('', '_2'))

    return merged

state_geometries = get_geometries(state, state_field)
county_geometries = get_geometries(county, county_field)
tribal1_geometries = get_geometries(tribal, tribal_field1)
tribal2_geometries = get_geometries(tribal, tribal_field2)
tribal3_geometries = get_geometries(tribal, tribal_field3)

# Stack all geometries
adds_geometries = pd.concat([state_geometries, county_geometries, tribal1_geometries, tribal2_geometries, tribal3_geometries])

If there are no rows to quit, message & set switch to disable remaining blocks

In [11]:
proceed = True

# If no geometries to add, quit
if adds_geometries.empty:
    proceed = False
    print("Found no new Declarations to add! We're done here.")

Found no new Declarations to add! We're done here.


If there are adds, go ahead with the dissolve

In [12]:
if proceed:

    adds_feature_collection = adds_geometries.spatial.to_feature_collection()

    remove_fields = ["OBJECTID", "Shape__Area", "Shape__Length", "SHAPE"]
    dissolve_fields = [f["name"] for f in dashboard_layer.properties.fields if not f["name"] in remove_fields]

    adds_dissolved = manage_data.dissolve_boundaries(adds_feature_collection, dissolve_fields=dissolve_fields)

...and finally add the features to the dashboard layer

In [13]:
if proceed:
    
    adds_features = adds_dissolved.query().features
    message = dashboard_layer.edit_features(adds_features)

    message